En esta parte del código, leemos el archivo JSON más pequeño para extraer los datos. El archivo en cuestión es el "5-core" de la página titulado como 'Amazon Fashion'. Extraemos el 'overall' para poder calcular el promedio del rating. Los datos necesarios para el cálculo de la similitud se obtienen del archivo de metadatos, que se lee más adelante. 



Codigo inicial

In [2]:
#Codigo para cargar los datos de las review en el sistema
import json
# Abre el archivo JSON en modo lectura
with open('AMAZON_FASHION.json', 'r') as archivo:
    # Inicializa una lista para almacenar los datos
    datos = []

    # Lee el archivo línea por línea
    for linea in archivo:
        # Parsea cada línea como un objeto JSON
        objeto_json = json.loads(linea)
        # Agrega el objeto JSON a la lista de datos
        datos.append(objeto_json)


Mediante este codigo lo que hacemos es calcular el promedio de calificaciones de cada producto, en nuestro solo aplicamos el promedio debido a que el sentiment score siempre va a ser 1

In [ ]:
promedio_rating_por_producto = {}

for objeto_json in datos:
    asin = objeto_json['asin']
    rating = objeto_json['overall']

    if asin in promedio_rating_por_producto:
        promedio_rating_por_producto[asin]['total_rating'] += rating
        promedio_rating_por_producto[asin]['total_reviews'] += 1
    else:
        promedio_rating_por_producto[asin] = {
            'total_rating': rating,
            'total_reviews': 1
        }

# Calcula el promedio de calificación para cada producto
for asin, datos_producto in promedio_rating_por_producto.items():
    promedio = datos_producto['total_rating'] / datos_producto['total_reviews']
    promedio_rating_por_producto[asin]['promedio_rating'] = promedio

# Imprime el resultado
for asin, datos_producto in promedio_rating_por_producto.items():
    print(f"Producto {asin}: Promedio de Rating = {datos_producto['promedio_rating']:.2f}")

El codigo nos permite cargar en un vector los productos a los cuales un cliente hizo una reseña, esto nos servira mas adelante para crear la lista recomendada para este usuario.
En otras palabras enlazar el cliente con una lista de productos el cual el cliente le hizo una reseña.


In [ ]:
productos_por_cliente = {}

# Lee el archivo línea por línea
for objeto_json  in datos:
    # Obtiene el cliente (reviewerID) de la revisión
    cliente = objeto_json['reviewerID']
    # Obtiene el producto (asin) de la revisión
    producto = objeto_json['asin']
    # Verifica si el cliente ya está en el diccionario
    if cliente in productos_por_cliente:
        productos_por_cliente[cliente].append(producto)
    else:
        productos_por_cliente[cliente] = [producto]

# Ahora, 'productos_por_cliente' contiene un diccionario que mapea cada cliente a una lista de productos revisados
# Puedes acceder a los productos revisados por cada cliente o realizar análisis adicional
for cliente, productos in productos_por_cliente.items():
    print(f"Cliente: {cliente}, Productos revisados: {', '.join(productos)}")
    # Supongamos que deseas imprimir los productos revisados por el cliente "A1D4G1SNUZWQOT"
cliente_especifico = "A1D4G1SNUZWQOT"

# Verifica si el cliente específico existe en el diccionario
if cliente_especifico in productos_por_cliente:
    productos_del_cliente = productos_por_cliente[cliente_especifico]
    print(f"Productos revisados por el cliente {cliente_especifico}: {', '.join(productos_del_cliente)}")
else:
    print(f"El cliente {cliente_especifico} no ha revisado ningún producto.")


Aqui es donde leemos el metadata de el archivo que elegimos para probar nuestro proyecto , el de AMAZON_FASHION debido a que necesitamos el also_viewed y el also_buy , que basicamente son una lista de prodcutos los cuales
la gente que visito/compro el producto actual, tambien visito(also_view) o compro(also_buy) esos productos

Carga en el sistema la lista de productos, como nosotros solo usamos el also_buy y el also_viewed solo cargamos esos datos

Se crearon dos campos aparte para el rating y el nivel de recomendacion.
Por ultimo se carga el rating , el promedio de calificaiones que calculamos anteriormente. 

In [ ]:
import json
# Abre el archivo JSON en modo lectura
with open('meta_AMAZON_FASHION.json', 'r') as archivo:
    # Inicializa una lista para almacenar los datos
    datos_meta = []

    # Lee el archivo línea por línea
    for linea in archivo:
        # Parsea cada línea como un objeto JSON
        objeto_json = json.loads(linea)
        
        # Extrae solo las columnas 'asin', 'also_buy' y 'also_viewed'
        datos_meta.append({
            'asin': objeto_json.get('asin', None),
            'also_buy': objeto_json.get('also_buy', None),
            'also_viewed': objeto_json.get('also_viewed', None),
            'rating':None,
            'Nivel_Recomendacion':None,
        })
# Actualiza los valores de rating en datos_meta
for dato in datos_meta:
    asin = dato['asin']
    if asin in promedio_rating_por_producto:
        dato['rating'] = promedio_rating_por_producto[asin]['promedio_rating']

# Imprime el resultado
for dato in datos_meta:
    print(dato)


Se intento hacer mediante el cossine pero como vemos genera vectores que ocupan mucho espacio en el sismtema, este metodo serviria para conjunto de datos pequeños en cambio para grandes conjuntos como manejamos nosotros es mejor usar otros metodos como el jaccard

In [ ]:
"""import numpy as np

def calcular_similitud(vector1, vector2):
    # Asegurarse de que los vectores tengan la misma longitud
    if len(vector1) != len(vector2):
        raise ValueError("Los vectores deben tener la misma longitud")

    # Calcular el producto punto entre los dos vectores
    producto_punto = np.dot(vector1, vector2)

    # Calcular la magnitud (norma) de cada vector
    norma_vector1 = np.linalg.norm(vector1)
    norma_vector2 = np.linalg.norm(vector2)

    # Verificar si una de las normas es cero
    if norma_vector1 == 0 or norma_vector2 == 0:
        return 0

    # Calcular la similitud utilizando la fórmula proporcionada
    similitud = producto_punto / (norma_vector1 * norma_vector2)

    return similitud
def crear_vector_binario_completo(datos, conjunto):
    # Crea un vector binario de ceros del tamaño de datos_meta
    vector_binario = [0] * len(datos)

    # Itera a través de los datos y asigna 1 a las posiciones correspondientes en el vector
    for i, dato in enumerate(datos):
        asin = dato['asin']
        if asin in conjunto:
            vector_binario[i] = 1

    return vector_binario

def combinar_sets(datos):
    # Crea un diccionario para almacenar los conjuntos combinados
    combined_sets = {}

    for dato in datos:
        asin = dato['asin']
        also_buy = dato['also_buy']
        also_viewed = dato['also_viewed']

        if asin not in combined_sets:
            # Si el asin no está en el diccionario, crea un conjunto vacío para él
            combined_sets[asin] = set()

        if also_buy:
            # Agrega los elementos de 'also_buy' al conjunto del producto
            combined_sets[asin].update(also_buy)

        if also_viewed:
            # Agrega los elementos de 'also_viewed' al conjunto del producto
            combined_sets[asin].update(also_viewed)

    return combined_sets

# Llama a la función para combinar los conjuntos
conjuntos_combinados = combinar_sets(datos_meta)
contador=0
# Ahora puedes acceder a los conjuntos combinados para cada producto por su 'asin'
for asin, conjunto in conjuntos_combinados.items():
    vector_binario = crear_vector_binario_completo(datos_meta, conjunto)
    
    # Imprime el ASIN y el vector binario resultante
    print(f'ASIN: {asin}')
    print('Conjunto combinado:', conjunto)
    print('Vector binario:', vector_binario)
    
    # Incrementa el contador
    contador += 1
    
    # Si hemos impreso los primeros 10 elementos, sal del bucle
    if contador >= 10:
        break
"""

Para convertir esa escala de 0 a 10 a una escala de 0 a 1, simplemente divide cada valor por 10. Aquí tienes la conversión:

0 en la escala de 0 a 10 equivale a 0 en la escala de 0 a 1.
3 en la escala de 0 a 10 equivale a 0.3 en la escala de 0 a 1.
5.5 en la escala de 0 a 10 equivale a 0.55 en la escala de 0 a 1.
8 en la escala de 0 a 10 equivale a 0.8 en la escala de 0 a 1.
10 en la escala de 0 a 10 equivale a 1 en la escala de 0 a 1.
Entonces, en la escala de 0 a 1, los valores se verían así:

Para el similarity score:

0 - 0.3 (poor)

0.25 - 0.55 (average)

0.5 - 0.8 (good)

0.7 - 1 (excellent)

Para el Product rating score cambia el rango (0-5):

0- 2.5 poor
2.3 - 3 average
2.8 -  4 good
3.8 - 5 excelentt


Este codigo Lo que realiza es la comparacion de similitud de todos los productos con todos los productos, por lo que tarda mucho tiempo, la idea de nuestro proyecto es realizar la comparacion solo de los productos con los cuales un cliente hizo la reseña por ejemplo
Cliente: A3DDWDH9PX2YX2, Productos revisados: 7106116521, B00J8WZBEG solo compararia con los productos 7106116521, B00J8WZBEG

fuzz.trimf es una  funcion dentro de la biblioteca fuzzywuzzy que se utiliza para definir una funcion de membresia triangular. 
Simi.universe representa al universo y el segundo argumennto es la funciopn de membresia triangular.
El primer nummero es de donde empieza a umentar m el segundo punto donde alcanza su valor maximo y el tercer valor es el punto donde la funcion comienza a disminuir hacia cero

Luego creamos nuestras reglas difusas con ctrl.rule , donde el primer argumento es la parte que define las conficiones que activaran la regla y el segundo argumento el resultado si se da dicha regla.
Por ultimo creamos nuestro sistema difuso.

Tambien en este bloque se uso el metodo de similitud de jaccard , el cual nos arroja un resultado entre 0-1. Basicamente se calcula la interseccion y union de dos conjuntos y si similiraty score es = intersection / union.
En nuestro caso lo aplicamos con el conjunto de also_view y also_buy de cada producto.


In [73]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
def defu(similitud,rate):
    try:
        # Crear las variables de entrada y salida difusa
        simi = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'simi')
        rating= ctrl.Antecedent(np.arange(0, 5.01, 0.01), 'rating')
        recomendacion = ctrl.Consequent(np.arange(0, 1, 0.1), 'recomendacion')

        # Definir funciones de membresía para las variables
        simi['poor'] = fuzz.trimf(simi.universe, [0, 0, 0.3])
        simi['average'] = fuzz.trimf(simi.universe, [0.25, 0.55, 0.8])
        simi['good'] = fuzz.trimf(simi.universe, [0.5, 0.8, 1])
        simi['excellent'] = fuzz.trimf(simi.universe, [0.7, 1, 1])
        rating['poor'] = fuzz.trimf(rating.universe, [0, 0, 2.5])
        rating['average'] = fuzz.trimf(rating.universe, [2.3, 3, 3.8])
        rating['good'] = fuzz.trimf(rating.universe, [2.8, 4, 4])
        rating['excellent'] = fuzz.trimf(rating.universe, [3.8, 5, 5])
        recomendacion['no recomendado'] = fuzz.trimf(recomendacion.universe, [0, 0, 0.25])
        recomendacion['probablemente recomendado'] = fuzz.trimf(recomendacion.universe, [0, 0.25, 0.5])
        recomendacion['recomendado'] = fuzz.trimf(recomendacion.universe, [0.25, 0.5, 0.75])
        recomendacion['muy recomendado'] = fuzz.trimf(recomendacion.universe, [0.5, 0.75, 1])
        # Reglas para la variable de salida "recomendacion"
        regla1 = ctrl.Rule(simi['poor'] & rating['poor'], recomendacion['no recomendado'])
        regla2 = ctrl.Rule(simi['poor'] & rating['average'], recomendacion['no recomendado'])
        regla3 = ctrl.Rule(simi['poor'] & rating['good'], recomendacion['probablemente recomendado'])
        regla4 = ctrl.Rule(simi['poor'] & rating['excellent'], recomendacion['recomendado'])

        regla5 = ctrl.Rule(simi['average'] & rating['poor'], recomendacion['no recomendado'])
        regla6 = ctrl.Rule(simi['average'] & rating['average'], recomendacion['no recomendado'])
        regla7 = ctrl.Rule(simi['average'] & rating['good'], recomendacion['recomendado'])
        regla8 = ctrl.Rule(simi['average'] & rating['excellent'], recomendacion['muy recomendado'])

        regla9 = ctrl.Rule(simi['good'] & rating['poor'], recomendacion['no recomendado'])
        regla10 = ctrl.Rule(simi['good'] & rating['average'], recomendacion['probablemente recomendado'])
        regla11 = ctrl.Rule(simi['good'] & rating['good'], recomendacion['recomendado'])
        regla12 = ctrl.Rule(simi['good'] & rating['excellent'], recomendacion['muy recomendado'])

        regla13 = ctrl.Rule(simi['excellent'] & rating['poor'], recomendacion['probablemente recomendado'])
        regla14 = ctrl.Rule(simi['excellent'] & rating['average'], recomendacion['recomendado'])
        regla15 = ctrl.Rule(simi['excellent'] & rating['good'], recomendacion['muy recomendado'])
        regla16 = ctrl.Rule(simi['excellent'] & rating['excellent'], recomendacion['muy recomendado'])


        # Crear el sistema de control difuso
        sistema_control = ctrl.ControlSystem([
            regla1, regla2, regla3, regla4, regla5, regla6, regla7, regla8,
            regla9, regla10, regla11, regla12, regla13, regla14, regla15, regla16
        ])

        # Simular el sistema con valores de entrada
        sistema = ctrl.ControlSystemSimulation(sistema_control)
        sistema.input['simi'] = similitud
        sistema.input['rating'] = rate
        # Obtener el resultado
        sistema.compute()
        resultado = sistema.output['recomendacion']

        if resultado <= 0.25:
            recomendacion_categoria = "No Recomendado"
        elif resultado <= 0.5:
            recomendacion_categoria = "Probablemente Recomendado"
        elif resultado <= 0.75:
            recomendacion_categoria = "Recomendado"
        else:
            recomendacion_categoria = "Muy Recomendado"
        print(f"Categoría de Recomendación: {recomendacion_categoria}")
    except AssertionError as e:
        print(f"Se produjo un error en la defuzzificación: {e} la similutud era {similitud} y el rate era {rate}")

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if not intersection or not union:
        return 0
    return intersection / union
   
def crear_lista(dato1):
    for dato in datos_meta:
        if dato['asin']==dato1:
            producto=dato
            break
    producto_set1=set()
    if producto['also_buy']:
        producto_set1.update(producto['also_buy'])
    if producto['also_viewed']:
        producto_set1.update(producto['also_viewed'])  
    
    for i, dato2 in enumerate(datos_meta):
        if dato2['asin']!=dato1:  # Evitar comparar el dato1 consigo mismo
            products_set2 = set()
            if dato2['also_buy']:
                products_set2.update(dato2['also_buy'])
            if dato2['also_viewed']:
                products_set2.update(dato2['also_viewed'])
            similarity = jaccard_similarity(producto_set1, products_set2)
            valor_dato2 = 0 if dato2['rating'] is None else dato2['rating']
            print(similarity)
            print(valor_dato2)
            defu(similarity,valor_dato2)


In [8]:
def recomendar_productos(cliente, productos_por_cliente):
    if cliente in productos_por_cliente:
        productos_revisados = productos_por_cliente[cliente]
        print(f"Productos revisados por el cliente {cliente}:")
        for producto in productos_revisados:
            crear_lista(producto)
    else:
        print(f"El cliente {cliente} no ha revisado ningún producto.")
    

In [ ]:
# Menú principal
while True:
    print("\nMenú:")
    print("1. Cargar ID de cliente")
    print("2. Salir")
    opcion = input("Selecciona una opción: ")
    if opcion == "1":
        cliente = input("Ingrese el ID del cliente: ")
        recomendar_productos(cliente, productos_por_cliente)
    elif opcion == "2":
        break
    else:
        print("Opción no válida. Por favor, seleccione una opción válida.")